In [ ]:
import requests 
import json
import pandas as pd
import ntpath
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import re
%matplotlib inline

In [ ]:
ALL_COMPETITOR_DATA_PATH = '../data/competitor_list_BubbleTea_ShavedIce.csv'
ZIP_CODE = '../data/UsZipsList.csv'
COMPETITORS_COUNT_PATH = '../data/num_competitor.csv'
MF_COUNT_PATH = '../data/MF_NmOfComments_AvgNote.csv'

ORIGINAL_COUNTY_DATA_PATH = '../data/county_data_num.csv'
GROUP2_DATA_PATH = '../data/group2/full_data_cleaned_RemoveNAN.csv'
MERGED_DATA_PATH = '../data/data_merged.csv'
MERGED_DATA_NOR_PATH = '../data/data_merged_nor.csv'

In [ ]:
def clean_zip_code(text):
    a = re.sub('^0', '', text)
    a = re.sub('\.0', '', a)
    a = re.sub('... ...', '0', a)
    a = re.sub('[A-Z].[A-Z]', '0', a)
    return re.sub('. .', '0', a)

In [ ]:
def get_competitors_count():
  resto = pd.read_csv(ALL_COMPETITOR_DATA_PATH)

  num_res_state = resto.groupby('state').agg({'id':'count', 'rating':'mean'}).sort_values('id', ascending=False)
  num_res_state.reset_index(inplace=True)

  num_res_city = resto.groupby('city').agg({'id':'count'}).sort_values('id', ascending=False)
  num_res_city.reset_index(inplace=True)

  zipcode = pd.read_csv(ZIP_CODE)

  num_res_zipcode = resto.groupby('zip code').agg({'id':'count'}).sort_values('id', ascending=False)
  num_res_zipcode.reset_index(inplace=True)
  num_res_zipcode['zip code'] = num_res_zipcode['zip code'].apply(clean_zip_code).astype('int64')

  num_res_zipcode['county'] = 0
  num_res_zipcode['city'] = 0
  num_res_zipcode['state'] = 0
  num_res_zipcode['population'] = 0

  for i in tqdm(range(num_res_zipcode.shape[0])):
  for j in range(zipcode.shape[0]):
    if num_res_zipcode.loc[i,'zip code'] == zipcode.loc[j,'zip']:
      num_res_zipcode.loc[i,'county'] = zipcode.loc[j,'county_name']
      num_res_zipcode.loc[i,'city'] = zipcode.loc[j,'city']
      num_res_zipcode.loc[i,'state'] = zipcode.loc[j,'state_name']
      num_res_zipcode.loc[i,'population'] = zipcode.loc[j,'population']
      continue

  num_res_zipcode = num_res_zipcode.rename(columns = {'id':'num_competitor'})
  
  num_res_zipcode.to_csv(COMPETITORS_COUNT_PATH)

In [ ]:
def get_MF_count():
  resto = pd.read_csv(ALL_COMPETITOR_DATA_PATH, index_col=0)
  resto = resto[resto['name']=='Meet Fresh']

  resto['zip code'] = resto['zip code'].apply(clean_zip_code).astype('int64')
  resto = resto.drop(resto[resto['zip code']==0].index)

  # add fips info
  zipcode = pd.read_csv(ZIP_CODE)
  data = pd.merge(resto,zipcode, how = 'left', left_on=['zip code'], right_on = ['zip']).set_index('county_fips').drop_duplicates(subset='id')

  data = data.groupby('county_fips').agg({"review count":"sum", "rating":"mean"})
  data.to_csv(MF_COUNT_PATH)

In [ ]:
def combien_data_group2():
  # 处理state id
  group_2 = pd.read_csv(GROUP2_DATA_PATH)
  pd.unique(group_2['state_id'])
  values = ['PR', 'VI', 'AK', 'HI']
  group_2 = group_2[group_2.state_id.isin(values) == False]
  pd.unique(group_2['state_id'])
  group_2['state_id'] = group_2['state_id'].replace(['MA', 'RI', 'NH', 'ME', 'VT', 'CT', 'NY', 'NJ', 'PA', 'DE', 'DC',
                                                   'VA', 'MD', 'WV', 'NC', 'SC', 'GA', 'FL', 'AL', 'TN', 'MS', 'KY',
                                                   'OH', 'IN', 'MI', 'IA', 'WI', 'MN', 'SD', 'ND', 'MT', 'IL', 'MO',
                                                   'KS', 'NE', 'LA', 'AR', 'OK', 'TX', 'CO', 'WY', 'ID', 'UT', 'AZ',
                                                   'NM', 'NV', 'CA', 'OR', 'WA'],
                                                  ['Massachusetts', 'Rhode Island', 'New Hampshire', 'Maine', 'Vermont',
                                                   'Connecticut', 'New York', 'New Jersey', 'Pennsylvania', 'Delaware',
                                                   'District of Columbia', 'Virginia', 'Maryland', 'West Virginia',
                                                   'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Alabama',
                                                   'Tennessee', 'Mississippi', 'Kentucky', 'Ohio', 'Indiana', 'Michigan',
                                                   'Iowa', 'Wisconsin', 'Minnesota', 'South Dakota', 'North Dakota',
                                                   'Montana', 'Illinois', 'Missouri', 'Kansas', 'Nebraska', 'Louisiana',
                                                   'Arkansas', 'Oklahoma', 'Texas', 'Colorado', 'Wyoming', 'Idaho', 'Utah',
                                                   'Arizona', 'New Mexico', 'Nevada', 'California', 'Oregon', 'Washington']
                                                  )
  
  zipcode = pd.read_csv(ZIP_CODE, index_col = 'zip')

  old = group_2.set_index('zip')
  old['county_fips'] = zipcode['county_fips']
  data = old.loc[old['county_fips'].notnull()]

  groupby = data.groupby(['state_id','county_name']).agg('mean').reset_index()
  groupby['county_fips'] = groupby['county_fips'].astype('int')
  groupby.rename(columns = {'county_fips':'FIPS'}, inplace=True)
  groupby = groupby.set_index('FIPS')

  count = data.groupby(['state_id','county_name']).agg('count')
  count.index=groupby.index

  data = groupby*count

  original_data = pd.read_csv(ORIGINAL_COUNTY_DATA_PATH, index_col = 'FIPS').drop(columns='Unnamed: 0', inplace=True)

  all = pd.merge(original_data, data, on='FIPS')

  all.to_csv(MERGED_DATA_PATH)

In [ ]:
def normalization():
  data = pd.read_csv(MERGED_DATA_PATH, index_col = 'FIPS')
  data_nor = data.groupby('State').transform(lambda x : (x-x.min())/(x.max()-x.min())).drop(columns = ['MF_exist'])
  header = data[['State', 'County', 'MF_exist']]
  result = pd.concat([header, data_nor], axis = 1)

  result.to_csv(MERGED_DATA_NOR_PATH)